In [1]:
import pandas as pd
import os
import hashlib
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
file_path="~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/CMLs_20110609_20110813.dat"

In [3]:
df = pd.read_table(file_path, sep="\s+", index_col=False)

In [4]:
df["Start_Concat"]=df["YStart"].astype(str)+" "+df["XStart"].astype(str)
df["End_Concat"]=df["YEnd"].astype(str)+" "+df["XEnd"].astype(str)

In [5]:
def convert_to_four_letter_hash(concat):
    h = hashlib.sha1(str.encode(concat))
    d = h.digest()
    s = ""
    for i in range(0,4):
        x = d[i] % 52
        if x >= 26:
            s += chr(ord('A') + x - 26)
        else:
            s += chr(ord('a') + x)
    return s

In [6]:
def convert_datetime_to_time(datetime):
    yyyy=datetime[:4]
    mm=datetime[4:6]
    dd=datetime[6:8]
    hh=datetime[-4:-2]
    ss=datetime[-2:]
    
    return(f"{dd}/{mm}/{yyyy} {hh}:{ss}")
    

In [7]:
df["ID"]=df["Start_Concat"].apply(convert_to_four_letter_hash).astype(str)+"-"+df["End_Concat"].apply(convert_to_four_letter_hash).astype(str)

In [8]:
df["Time"]=df["DateTime"].astype(str).apply(convert_datetime_to_time)

In [9]:
ids=df.ID.unique()

In [19]:
df["PowerTLTMmax[dBm]"]=5
df["PowerTLTMmin[dBm]"]=3
df.rename(columns={"Pmin": "PowerRLTMmin[dBm]", "Pmax": "PowerRLTMmax[dBm]"}, inplace=True)
df.dropna(inplace=True)

In [21]:
root="~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/CellEnMon/datasets/dme/09062011_13082011/processed"
columns=["Time","PowerTLTMmax[dBm]","PowerTLTMmin[dBm]","PowerRLTMmax[dBm]","PowerRLTMmin[dBm]"]
for id in ids[:10]:
    T,R=id.split("-")
    d=df[df.ID==id]
    d.to_csv(f'{root}/{T}_{d.YStart.iloc[0]}_{d.XStart.iloc[0]}_{R}_{d.YEnd.iloc[0]}_{d.XStart.iloc[0]}.csv',index=False, columns=columns)
    

In [20]:
df.head()

,Frequency,DateTime,PowerRLTMmin[dBm],PowerRLTMmax[dBm],PathLength,XStart,YStart,XEnd,YEnd,ID,Start_Concat,End_Concat,Time,PowerTLTMmax[dBm],PowerTLTMmin[dBm]
0,39.263,201106090815,-61.0,-59.0,3.12961,4.75907,52.41104,4.78890,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09/06/2011 08:15,5,3
1,39.172,201106090815,-53.0,-51.0,2.89575,5.04335,52.51464,5.07860,52.49999,VUDJ-ImjW,52.51464 5.04335,52.49999 5.0786,09/06/2011 08:15,5,3
2,39.347,201106090815,-50.0,-50.0,1.14711,4.96120,52.50239,4.97624,52.50708,WbQT-MVOn,52.50239 4.9612,52.50708 4.97624,09/06/2011 08:15,5,3
3,39.172,201106090815,-48.0,-48.0,4.26850,4.75550,52.44724,4.81687,52.43914,VCRy-sbeU,52.44724 4.7555,52.43914 4.81687,09/06/2011 08:15,5,3
4,37.069,201106090815,-53.0,-52.0,1.83034,4.81166,52.39787,4.78949,52.40718,NubJ-vcWP,52.39787 4.81166,52.40718 4.78949,09/06/2011 08:15,5,3
